In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers torch tqdm Pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [ ]:
import os
import pandas as pd
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import torch
from tqdm import tqdm
import concurrent.futures

csv_path = "path/to/your/dataset.csv"
base_image_dir = "path/to/images/"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Using device: {device}")

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)
model.eval()

df = pd.read_csv(csv_path)

def generate_caption(image_path):
    try:
        image = Image.open(image_path).convert("RGB")
        inputs = processor(images=image, return_tensors="pt").to(device)
        with torch.no_grad():
            output = model.generate(**inputs)
        caption = processor.decode(output[0], skip_special_tokens=True)
        return caption
    except Exception as e:
        print(f"⚠️ Error processing {image_path}: {e}")
        return ""

def process_images_parallel(image_paths):
    captions = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
        results = list(tqdm(executor.map(generate_caption, image_paths), total=len(image_paths), desc="🖼 Generating captions"))
        captions.extend(results)
    return captions

chunk_size = 100
image_paths = df["image_path"].apply(lambda x: os.path.join(base_image_dir, x.replace("\\", "/"))).tolist()
captions = []

total_chunks = len(image_paths) // chunk_size + (1 if len(image_paths) % chunk_size > 0 else 0)
print(f"Total chunks: {total_chunks}")

for i in range(0, len(image_paths), chunk_size):
    chunk = image_paths[i:i+chunk_size]
    chunk_captions = process_images_parallel(chunk)
    captions.extend(chunk_captions)
    remaining_chunks = total_chunks - (i // chunk_size + 1)
    print(f"Processed chunk {i // chunk_size + 1}/{total_chunks}, Remaining chunks: {remaining_chunks}")

df["generated_caption"] = captions
df.to_csv("path/to/output.csv", index=False)
print("✅ Captions added and CSV saved.")


🚀 Using device: cuda
Total chunks: 185


🖼 Generating captions: 100%|██████████| 100/100 [00:30<00:00,  3.31it/s]


Processed chunk 1/185, Remaining chunks: 184


🖼 Generating captions: 100%|██████████| 100/100 [00:29<00:00,  3.41it/s]


Processed chunk 2/185, Remaining chunks: 183


...

Processed chunk 183/185, Remaining chunks: 2


🖼 Generating captions: 100%|██████████| 100/100 [00:32<00:00,  3.09it/s]


Processed chunk 184/185, Remaining chunks: 1


🖼 Generating captions: 100%|██████████| 16/16 [00:04<00:00,  3.22it/s]


Processed chunk 185/185, Remaining chunks: 0
✅ Captions added and CSV saved.


In [ ]:
!ls


drive  sample_data


In [ ]:
!rm torch.py torch.pyc


rm: cannot remove 'torch.py': No such file or directory
rm: cannot remove 'torch.pyc': No such file or directory


In [ ]:
!pip uninstall -y torch torchvision torchaudio
!pip uninstall -y transformers

!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install transformers


Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
Found existing installation: transformers 4.51.3
Uninstalling transformers-4.51.3:
  Successfully uninstalled transformers-4.51.3
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 90.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 114.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 3.9 MB/s eta 0:00:0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 109.9 MB/s eta 0:00:00
ERROR: Operation cancelled by user
^C


In [ ]:
import os
import pandas as pd
from transformers import AutoProcessor, GitForCausalLM
from PIL import Image
import torch
from tqdm import tqdm
import concurrent.futures

csv_path = "path/to/your/dataset.csv"
base_image_dir = "path/to/images/"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Using device: {device}")

processor = AutoProcessor.from_pretrained("microsoft/git-base")
model = GitForCausalLM.from_pretrained("microsoft/git-base").to(device)
model.eval()

df = pd.read_csv(csv_path)

def generate_caption(image_path):
    try:
        image = Image.open(image_path).convert("RGB")
        pixel_values = processor(images=image, return_tensors="pt").pixel_values.to(device)
        input_ids = processor(text="generate caption:", return_tensors="pt").input_ids.to(device)

        with torch.no_grad():
            generated_ids = model.generate(pixel_values=pixel_values, input_ids=input_ids, max_length=50)

        caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
        return caption
    except Exception as e:
        print(f"⚠️ Error processing {image_path}: {e}")
        return ""

def process_images_parallel(image_paths):
    captions = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
        results = list(tqdm(executor.map(generate_caption, image_paths), total=len(image_paths), desc="🖼 Generating captions"))
        captions.extend(results)
    return captions

chunk_size = 100
image_paths = df["image_path"].apply(lambda x: os.path.join(base_image_dir, x.replace("\\", "/"))).tolist()
captions = []

total_chunks = len(image_paths) // chunk_size + (1 if len(image_paths) % chunk_size > 0 else 0)
print(f"Total chunks: {total_chunks}")

for i in range(0, len(image_paths), chunk_size):
    chunk = image_paths[i:i+chunk_size]
    chunk_captions = process_images_parallel(chunk)
    captions.extend(chunk_captions)
    remaining_chunks = total_chunks - (i // chunk_size + 1)
    print(f"Processed chunk {i // chunk_size + 1}/{total_chunks}, Remaining chunks: {remaining_chunks}")

df["generated_caption"] = captions
df.to_csv("path/to/output_git_base.csv", index=False)
print("✅ Captions added and CSV saved.")


🚀 Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/503 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/453 [00:00<?, ?B/s]

...

Processed chunk 183/185, Remaining chunks: 2


🖼 Generating captions: 100%|██████████| 100/100 [00:21<00:00,  4.60it/s]


Processed chunk 184/185, Remaining chunks: 1


🖼 Generating captions: 100%|██████████| 16/16 [00:05<00:00,  2.67it/s]


Processed chunk 185/185, Remaining chunks: 0
✅ Captions added and CSV saved.


In [ ]:
import os
import pandas as pd
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
from PIL import Image, UnidentifiedImageError
import torch
from tqdm import tqdm
import concurrent.futures

csv_path = "path/to/your/dataset.csv"
image_folder = "path/to/images"
output_path = "path/to/output.csv"

df = pd.read_csv(csv_path)

def is_valid_image(unique_id):
    image_filename = f"{unique_id}.jpg"
    image_path = os.path.join(image_folder, image_filename)
    if not os.path.exists(image_path):
        return False
    try:
        img = Image.open(image_path)
        img.verify()
        return True
    except (UnidentifiedImageError, OSError):
        return False

valid_ids = []
for uid in tqdm(df["unique_id"], desc="Checking images"):
    if is_valid_image(uid):
        valid_ids.append(uid)
    if len(valid_ids) >= 20000:
        break

df = df[df["unique_id"].isin(valid_ids)].copy()
df.reset_index(drop=True, inplace=True)

model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
processor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

def generate_caption(image_path):
    try:
        image = Image.open(image_path).convert("RGB")
        pixel_values = processor(images=image, return_tensors="pt").pixel_values.to(device)
        with torch.no_grad():
            output_ids = model.generate(pixel_values, max_length=50, num_beams=4)
        caption = tokenizer.decode(output_ids[0], skip_special_tokens=True).strip()
        return caption
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return ""

def process_images_parallel(image_paths):
    captions = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
        results = list(tqdm(executor.map(generate_caption, image_paths), total=len(image_paths), desc="Generating captions"))
        captions.extend(results)
    return captions

image_paths = df["unique_id"].apply(lambda uid: os.path.join(image_folder, f"{uid}.jpg")).tolist()

chunk_size = 100
captions = []

total_chunks = len(image_paths) // chunk_size + (1 if len(image_paths) % chunk_size > 0 else 0)

for i in range(0, len(image_paths), chunk_size):
    chunk = image_paths[i:i + chunk_size]
    chunk_captions = process_images_parallel(chunk)
    captions.extend(chunk_captions)

df["generated_caption"] = captions
df.to_csv(output_path, index=False)
print("Captions saved.")


📥 Loading CSV...
🔎 Validating images...


🔍 Checking images: 100%|██████████| 40945/40945 [57:26<00:00, 11.88it/s]


✅ Found 7736 valid images.
📦 Loading model...


Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "architectures": [
    "ViTModel"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 224,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "pooler_act": "tanh",
  "pooler_output_size": 768,
  "qkv_bias": true,
  "torch_dtype": "float32",
  "transformers_version": "4.51.3"
}

Config of the decoder: <class 'transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel'> is overwritten by shared decoder config: GPT2Config {
  "activation_function": "gelu_new",
  "add_cross_attention": true,
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "decoder_start_to

🚀 Using device: cuda
🗂 Building image paths...
🧩 Total chunks: 78
🔄 Processing chunk 1 of 78...


...

✅ Chunk 76 done.
🔄 Processing chunk 77 of 78...


🖼 Generating captions: 100%|██████████| 100/100 [00:35<00:00,  2.83it/s]


✅ Chunk 77 done.
🔄 Processing chunk 78 of 78...


🖼 Generating captions: 100%|██████████| 36/36 [00:12<00:00,  2.79it/s]


✅ Chunk 78 done.
✅ Captions saved to file: /content/drive/MyDrive/ColabNotebooks_/dataset_thesis/labels/dataset_with_generated_captions_git_20000.csv


In [ ]:
# !pip install transformers torch torchvision
# !pip install ftfy  # для корректной обработки текста
# !pip install git+https://github.com/rmokady/CLIP_prefix_caption.git
!pip install git+https://github.com/openai/CLIP.git



  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-nmrupw96
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-nmrupw96
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# !pip install ftfy
!pip install transformers


  Using cached transformers-4.51.3-py3-none-any.whl.metadata (38 kB)
Using cached transformers-4.51.3-py3-none-any.whl (10.4 MB)


In [ ]:
!pip install --upgrade transformers


In [ ]:
!pip install --upgrade transformers


In [ ]:
!pip install ftfy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.5 MB/s eta 0:00:00


In [ ]:
from transformers import AutoProcessor, AutoModelForCausalLM
import torch
from PIL import Image, UnidentifiedImageError
import ftfy
from tqdm import tqdm
import concurrent.futures
import pandas as pd
import os

csv_path = "path/to/your/dataset.csv"
image_folder = "path/to/images"
output_csv_path = "path/to/output.csv"

git_processor = AutoProcessor.from_pretrained("microsoft/git-base")
git_model = AutoModelForCausalLM.from_pretrained("microsoft/git-base")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
git_model.to(device)

print(f"Using device: {device}")

df = pd.read_csv(csv_path)

def is_valid_image(unique_id):
    image_filename = f"{unique_id}.jpg"
    image_path = os.path.join(image_folder, image_filename)
    if not os.path.exists(image_path):
        return False
    try:
        img = Image.open(image_path)
        img.verify()
        return True
    except (UnidentifiedImageError, OSError):
        return False

def generate_caption(image_path):
    try:
        image = Image.open(image_path).convert("RGB")
        inputs = git_processor(images=image, return_tensors="pt").to(device)
        pixel_values = inputs.pixel_values
        generated_ids = git_model.generate(pixel_values=pixel_values, max_length=50)
        generated_caption = git_processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
        return ftfy.fix_text(generated_caption)
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return ""

def process_images_parallel(image_paths):
    captions = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
        results = list(tqdm(executor.map(generate_caption, image_paths), total=len(image_paths), desc="Generating captions"))
        captions.extend(results)
    return captions

valid_entries = []
for _, row in tqdm(df.iterrows(), total=len(df), desc="Checking valid images"):
    uid = row["unique_id"]
    if is_valid_image(uid):
        valid_entries.append(row)
    if len(valid_entries) >= 20000:
        break

valid_df = pd.DataFrame(valid_entries)
valid_df = valid_df.reset_index(drop=True)

image_paths = [os.path.join(image_folder, f"{uid}.jpg") for uid in valid_df["unique_id"]]

chunk_size = 100
captions = []
total_chunks = len(image_paths) // chunk_size + (1 if len(image_paths) % chunk_size > 0 else 0)
print(f"Total chunks: {total_chunks}")

for i in range(0, len(image_paths), chunk_size):
    chunk = image_paths[i:i + chunk_size]
    chunk_captions = process_images_parallel(chunk)
    captions.extend(chunk_captions)
    print(f"Processed chunk {i // chunk_size + 1}/{total_chunks}")

valid_df["generated_caption"] = captions
valid_df.to_csv(output_csv_path, index=False)
print("Captions generated and saved.")


preprocessor_config.json:   0%|          | 0.00/503 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/453 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

...

Processed chunk 76/78


🖼 Generating captions: 100%|██████████| 100/100 [00:23<00:00,  4.30it/s]


Processed chunk 77/78


🖼 Generating captions: 100%|██████████| 36/36 [00:08<00:00,  4.10it/s]


Processed chunk 78/78
✅ Captions generated and saved.
